In [1]:
import numpy as np
from tqdm.auto import tqdm
from src.quantum_states import create_ghz_density_matrix
from src.bell_sampling import (
    bell_measurement_probs,
    sample_bell_measurement_binArr,
)

from src.paulis import generate_all_Ps, calc_pauli_vec, generate_all_Ps_stacked

* we want fast routines to compute tr(PxP |B_ij><B_ij|)
* for P some pauli op (multi qubit)
* B_ij some bell state (multi qubit)
* given: pauli string XYZ means XYZ XYZ is applied to 2-copy system a1,a2,a3,b1,b2,b3
* ith index of B_ij

In [2]:
# paulis
px = np.array([[0, 1], [1, 0]])
py = np.array([[0, -1j], [1j, 0]])
pz = np.array([[1, 0], [0, -1]])
pi = np.eye(2)

paulis = {"X": px, "Y": py, "Z": pz, "I": pi}

In [3]:
one = np.array([[0], [1]])
zero = np.array([[1], [0]])

# bell states
basis_states = {
    "00": np.kron(zero, zero),
    "01": np.kron(zero, one),
    "10": np.kron(one, zero),
    "11": np.kron(one, one),
}

phip = (basis_states["00"] + basis_states["11"]) / np.sqrt(2)
phim = (basis_states["00"] - basis_states["11"]) / np.sqrt(2)
psip = (basis_states["01"] + basis_states["10"]) / np.sqrt(2)
psim = (basis_states["01"] - basis_states["10"]) / np.sqrt(2)


bell_states = {
    "phi+": phip,  # 1
    "psi+": psip,  # 2
    "phi-": phim,  # 3
    "psi-": psim,  # 4
}
bell_states_order = ["phi+", "psi+", "phi-", "psi-"]

In [4]:
mapping1 = dict()
for plabel, p in paulis.items():
    for bidx, blabel in enumerate(bell_states_order):
        b = bell_states[blabel]
        val = np.trace(np.kron(p, p) @ np.outer(b, b.conj().T))
        val = round(np.real(val).item())
        # val = np.dot(b.conj().T, np.kron(p, p) @ b)
        mapping1[(plabel, bidx)] = val

        blabel = blabel.replace("phi", "Φ")
        blabel = blabel.replace("psi", "Ψ")
        print(f"tr({plabel}{plabel} |{blabel}><{blabel}|) = {val}")

tr(XX |Φ+><Φ+|) = 1
tr(XX |Ψ+><Ψ+|) = 1
tr(XX |Φ-><Φ-|) = -1
tr(XX |Ψ-><Ψ-|) = -1
tr(YY |Φ+><Φ+|) = -1
tr(YY |Ψ+><Ψ+|) = 1
tr(YY |Φ-><Φ-|) = 1
tr(YY |Ψ-><Ψ-|) = -1
tr(ZZ |Φ+><Φ+|) = 1
tr(ZZ |Ψ+><Ψ+|) = -1
tr(ZZ |Φ-><Φ-|) = 1
tr(ZZ |Ψ-><Ψ-|) = -1
tr(II |Φ+><Φ+|) = 1
tr(II |Ψ+><Ψ+|) = 1
tr(II |Φ-><Φ-|) = 1
tr(II |Ψ-><Ψ-|) = 1


In [5]:
n = 3
ghz = create_ghz_density_matrix(n)
all_paulis = generate_all_Ps(n)
labels, all_paulis_stacked = generate_all_Ps_stacked(n, all_paulis)

In [6]:
pauli_vec_exact = calc_pauli_vec(ghz, all_paulis_stacked)

In [7]:
pauli_vec_exact

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
       -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,
        0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.])

In [8]:
exp_exact = {l.item():up.item()**2 for up,l in zip(pauli_vec_exact, labels)}

In [9]:
%pdb on
def compute_pauli_expectation(bell_samples, pauli_set, show_progress=True):
    """Compute the expectation value of Pauli operators using Monte Carlo approximation.
    For all pauli operators P in pauli_set, we compute
    E[tr(PxP |B_{a1,a2,a3,b1,b2,b3}><B_{a1,a2,a3,b1,b2,b3}|)]
    which is an estimator for
    |tr(P rho)|^2
    """
    # Compute E[tr(PxP |B_{a1,a2,a3,b1,b2,b3}><B_{a1,a2,a3,b1,b2,b3}|)]
    # use that P and B factorize: for example if we have 3 qubits and P = XZY
    # PxP = XZYXZY so if we consider the subsystem (a1,b1)
    # we have XX |B_{a1,b1}><B_{a1,b1}|
    # and for (a2,b2): ZZ |B_{a2,b2}><B_{a2,b2}|
    # and for (a3,b3): YY |B_{a3,b3}><B_{a3,b3}|
    # expectation value factorizes: tr(PxP |B_{a1,a2,a3,b1,b2,b3}><B_{a1,a2,a3,b1,b2,b3}|)
    # = tr(XX |B_{a1,b1}><B_{a1,b1}|) tr(ZZ |B_{a2,b2}><B_{a2,b2}|) tr(YY |B_{a3,b3}><B_{a3,b3}|)
    # for 2qubit cases we precomputed the signs of every bell state with every pauli operator
    # bell state order is Φ+, Ψ+, Φ-, Ψ-

    # generated above in mapping1
    mapping = {
        ("X", 0): 1,
        ("X", 1): 1,
        ("X", 2): -1,
        ("X", 3): -1,
        ("Y", 0): -1,
        ("Y", 1): 1,
        ("Y", 2): 1,
        ("Y", 3): -1,
        ("Z", 0): 1,
        ("Z", 1): -1,
        ("Z", 2): 1,
        ("Z", 3): -1,
        ("I", 0): 1,
        ("I", 1): 1,
        ("I", 2): 1,
        ("I", 3): 1,
    }

    nqubits = len(bell_samples[0]) // 2
    # bell_states = build_full_bell_generation_circuit(nqubits)
    expectations = {}

    pauli_iterator = (
        tqdm(
            pauli_set.items(),
            desc="Computing expectations",
            total=len(pauli_set),
            leave=False,
        )
        if show_progress
        else pauli_set.items()
    )

    # convert bell measurement bit string to 0,1,2,3 number indicating bell state
    bidx_meas = 2*bell_samples[:, :nqubits] + bell_samples[:, nqubits:]

    for label, _ in pauli_iterator:
        vals = []
        # PP = xp.kron(pauli, pauli) # dont need this anymore
        sample_iterator = (
            tqdm(bell_samples, desc="Computing samples", leave=False)
            if show_progress
            else bell_samples
        )

        for midx, meas in enumerate(sample_iterator):
            # Compute the expectation value of the Pauli operator
            # meas is bitstring of length 2*nqubits
            trPPBij_lookup = 1
            for qidx in range(nqubits):
                plabel = label[qidx]
                bidx = 2*meas[qidx] + meas[nqubits+qidx] # bell state index for sample midx
                trPPBij_lookup *= mapping[(plabel, bidx)]

            vals.append(trPPBij_lookup)
        expectations[label] = np.mean(vals)
        # print(label, expectations[label], vals)

    return expectations



Automatic pdb calling has been turned ON


In [10]:
N = 1000
meas_probs = bell_measurement_probs(ghz)
meas = sample_bell_measurement_binArr(ghz, num_samples=N, probs=meas_probs)

exp_mc = compute_pauli_expectation(meas, all_paulis, show_progress=False)

In [11]:
# compare exp_naive and exp_mc
residuals2 = []
for plabel in all_paulis.keys():
    residuals2.append((exp_exact[plabel] - exp_mc[plabel]) ** 2)

print(f"Mean squared residuals between exp_naive and exp_mc: {np.mean(residuals2)}")


Mean squared residuals between exp_naive and exp_mc: 0.001104
